In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download the punkt_tab data
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# Tokenize texts – you might want to apply further preprocessing in practice.
df['tokens'] = df['text'].apply(nltk.word_tokenize)

# Train a Word2Vec model on your tokenized texts
w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Example: Find similar words to 'love' (if present in your corpus)
print(w2v_model.wv.most_similar('author', topn=5))

[('text', 0.06799043715000153), ('two', 0.03362187743186951), ('Another', 0.009388835169374943), ('is', 0.008315935730934143), ('sample', 0.0045030261389911175)]


In [ ]:
import spacy

# Load an English model – you may need to download it first
nlp = spacy.load("en_core_web_sm")

# Apply NER on the text of the first document as an example
doc = nlp(df.loc[0, 'text'])
for ent in doc.ents:
    print(ent.text, ent.label_)


In [103]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
import spacy

# Download required nltk data (only needed once)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [122]:
import os
import glob
import yaml
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora

# Download required NLTK resources (only needed once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


def load_story(filename):
    """
    Load a text file with YAML front matter.
    Returns a tuple (metadata, text) where metadata is a dictionary and text is the content.
    """
    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()
    if content.startswith('---'):
        # Split the content into YAML front matter and main text.
        parts = content.split('---', 2)
        metadata = yaml.safe_load(parts[1])
        text = parts[2].strip()
    else:
        metadata = {}
        text = content.strip()
    return metadata, text

# Specify the directory containing your .txt files.
file_dir = 'texts/'  # <-- Change this to your actual folder path
files = glob.glob(os.path.join(file_dir, '*.txt'))

# Create a list to hold all documents with metadata.
data = []
for file in files:
    metadata, text = load_story(file)
    # Provide defaults if any metadata is missing.
    author = metadata.get('author', 'Unknown')
    year = metadata.get('year', None)
    title = metadata.get('title', os.path.basename(file))
    data.append({
        'author': author,
        'year': year,
        'title': title,
        'text': text
    })

# Create a DataFrame from the loaded data.
df = pd.DataFrame(data)
print("DataFrame Loaded:")
print(df.head())

# Tokenize the text.
df['tokens'] = df['text'].apply(word_tokenize)

#######################################
# Stopword and Punctuation Removal
#######################################
# Load the default English stopwords from NLTK.
default_stopwords = set(stopwords.words('english'))
# Create a set of punctuation characters.
punctuation_stopwords = set(string.punctuation)
# Define your custom stopwords here.
custom_stopwords = {"--", "editor", "page", "michael_j._pettid_", "_-_Silvery_World_and_Other_Stories__Anthology_of_Korean_Literature-Cornell_University_Press_".lower(), "2019", ".pdf", "'", "`", "“", "”", "’", "…","``","''","n't","'s","ch","oe"}  # Modify or extend as needed.
# Combine all stopwords.
all_stopwords = default_stopwords.union(punctuation_stopwords).union(custom_stopwords)

def clean_tokens(tokens, stopwords=all_stopwords):
    """
    Remove stopwords (default, punctuation, and custom) from a list of tokens.
    """
    return [token for token in tokens if token.lower() not in stopwords]

# Apply cleaning function to the tokenized texts.
df['clean_tokens'] = df['tokens'].apply(clean_tokens)

print("\nCleaned Tokens:")
print(df[['title', 'clean_tokens']])

#######################################
# Lemmatization
#######################################
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Lemmatize each token in the list."""
    return [lemmatizer.lemmatize(token) for token in tokens]

# Apply lemmatization.
df['lemmatized_tokens'] = df['clean_tokens'].apply(lemmatize_tokens)

print("\nLemmatized Tokens:")
print(df[['title', 'lemmatized_tokens']])

#######################################
# Creating Dictionary and Corpus for LDA
#######################################
# Build the corpus from the lemmatized tokens.
texts = df['lemmatized_tokens'].tolist()

# Create a dictionary representation of the documents.
id2word = corpora.Dictionary(texts)

# (Optional) Filter out tokens that are too rare or too common.
# id2word.filter_extremes(no_below=2, no_above=0.5)

# Create a corpus: a list of bag-of-words representations for each document.
corpus = [id2word.doc2bow(text) for text in texts]

# Output basic info for verification.
print("Number of unique tokens:", len(id2word))
print("Number of documents:", len(corpus))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


DataFrame Loaded:
          author  year             title  \
0     Kim Yujong  1936  The White Rabbit   
1     Yi T'aejun  1936             Crows   
2      Ch'oe Yun  1992  The Gray Snowman   
3      Yi Munyol  1979    The Old Hatter   
4  Ch’oe Chansik  1914         The Shore   

                                                text  
0  THE WHITE RABBIT \nNight and day I had that ra...  
1  CROWS \nThe lamp was new and the glass shade d...  
2  THE GRAY SNOWMAN \nThe events of that period a...  
3  THE OLD HATTER \nThe first encounter between o...  
4  ----- PAGE 151: Michael_J._Pettid_(editor)_-_S...  

Cleaned Tokens:
                               title  \
0                   The White Rabbit   
1                              Crows   
2                   The Gray Snowman   
3                     The Old Hatter   
4                          The Shore   
5                 Another Man's Room   
6                       Im Kkŏkchŏng   
7                             Lizard   
8         

In [123]:
#@title Old Data Prepearation
# #######################################
# # 1. Data Storage and Preparation
# #######################################
# # Here we store our texts along with metadata in a DataFrame.
# # This structure helps maintain contextual information (like author and year)
# # which is essential for historical and comparative analysis in digital humanities.
# data = [
#     {
#         'author': 'Kim Yuna',
#         'year': 1905,
#         'text': (
#             "In ancient Korea, wisdom and poetry flowed through the words of scholars.This text captures the essence of a time when language was both a conduit of tradition and a force of innovation. Every sentence echoes the rich history and evolving literary expression."
#         )
#     },
#     {
#         'author': 'Lee Minho',
#         'year': 1920,
#         'text': (
#             ""
#         )
#     },
#     # Add additional documents as needed.
# ]


# df = pd.DataFrame(data)

# # Tokenize texts
# df['tokens'] = df['text'].apply(nltk.word_tokenize)
# texts = df['tokens'].tolist()
# # Usefulness:
# # - Storing texts with metadata enables you to filter or group data by time period, author, etc.
# # - It supports longitudinal studies of language, themes, and cultural shifts.

In [124]:
#######################################
# 2. TF-IDF Vectorization
#######################################
# TF-IDF (Term Frequency-Inverse Document Frequency) weighs words based on their importance in a document versus the entire corpus.
# In a digital humanities context, this helps to highlight key terms that define the thematic essence of each work.
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])
print("TF-IDF matrix shape:", tfidf_matrix.shape)

TF-IDF matrix shape: (28, 1000)


In [160]:
#######################################
# 3. LDA for Topic Modeling
#######################################
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,            # Adjust number of topics as needed
                                           random_state=100,
                                           update_every=1,
                                           chunksize=25,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Display the topics with the top keywords
print("LDA Topics:")
for idx, topic in lda_model.print_topics(num_words=10):
    print(f"Topic {idx}: {topic}")

LDA Topics:
Topic 8: 0.000*"would" + 0.000*"one" + 0.000*"time" + 0.000*"could" + 0.000*"like" + 0.000*"day" + 0.000*"back" + 0.000*"man" + 0.000*"mother" + 0.000*"people"
Topic 24: 0.000*"said" + 0.000*"would" + 0.000*"could" + 0.000*"one" + 0.000*"like" + 0.000*"people" + 0.000*"mother" + 0.000*"time" + 0.000*"day" + 0.000*"woman"
Topic 27: 0.000*"like" + 0.000*"would" + 0.000*"could" + 0.000*"day" + 0.000*"said" + 0.000*"one" + 0.000*"time" + 0.000*"mother" + 0.000*"go" + 0.000*"man"
Topic 13: 0.000*"would" + 0.000*"like" + 0.000*"could" + 0.000*"time" + 0.000*"one" + 0.000*"come" + 0.000*"said" + 0.000*"go" + 0.000*"day" + 0.000*"man"
Topic 26: 0.000*"would" + 0.000*"one" + 0.000*"day" + 0.000*"time" + 0.000*"like" + 0.000*"could" + 0.000*"man" + 0.000*"come" + 0.000*"even" + 0.000*"woman"
Topic 14: 0.000*"would" + 0.000*"mother" + 0.000*"like" + 0.000*"time" + 0.000*"could" + 0.000*"one" + 0.000*"day" + 0.000*"man" + 0.000*"woman" + 0.000*"old"
Topic 11: 0.000*"could" + 0.000*"wou

In [161]:
#######################################
# 5. Visualizing Topics with pyLDAvis
#######################################
import pyLDAvis
import pyLDAvis.gensim
# Enable notebook mode for pyLDAvis (if using Jupyter Notebook)
pyLDAvis.enable_notebook()

# Prepare the visualization data
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

# Display the visualization (In a Jupyter environment, 'vis' will render the interactive visualization)
vis

TypeError: Object of type complex128 is not JSON serializable

PreparedData(topic_coordinates=                        x                                               y  \
topic                                                                       
10    -0.390120+0.000000j -9.853514e-02+0.000000e+                    00j   
12    -0.320997+0.000000j  1.228818e-02+0.000000e+                    00j   
25    -0.388590+0.000000j -1.498085e-01+0.000000e+                    00j   
3     -0.262209+0.000000j  1.439123e-01+0.000000e+                    00j   
23    -0.208199+0.000000j  2.385258e-01+0.000000e+                    00j   
16    -0.211050+0.000000j -1.891188e-02+0.000000e+                    00j   
19     0.079399+0.000000j  3.012885e-02+0.000000e+                    00j   
1      0.088481+0.000000j  9.472305e-07+0.000000e+                    00j   
28     0.085946+0.000000j  2.312006e-03+0.000000e+                    00j   
15     0.089236+0.000000j -1.321066e-02+0.000000e+                    00j   
5      0.086828+0.000000j -1.428344e-02+0.000

In [162]:
#######################################
# 4. Evaluating Topic Coherence
#######################################
# Coherence measures how semantically interpretable the topics are.
# It computes the average similarity between words in a topic.
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3172830656486886


In [138]:
#######################################
# 4. Clustering with K-Means
#######################################
# Clustering groups texts with similar content, potentially revealing sub-genres or periods.
# This is particularly useful for uncovering hidden structures within the corpus that correlate with historical or cultural shifts.
num_clusters = 2  # Adjust based on corpus diversity
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(tfidf_matrix)
df['cluster'] = clusters
print("\nDocument Clusters:")
print(df[['author', 'year', 'cluster']].sort_values(by=['year']))


Document Clusters:
                           author  year  cluster
10                       Yi Injik  1908        0
4                   Ch’oe Chansik  1914        0
25                   Hyon Chingon  1924        0
23                     Kim Tongin  1925        1
26                      Yi Kiyŏng  1925        0
11                  Cho Myŏng-hŭi  1927        0
18  Yong kwa yong ŭi taegyŏk-chŏn  1928        0
0                      Kim Yujong  1936        1
1                      Yi T'aejun  1936        1
21                      Yi Hyosok  1936        1
17                        Yi Sang  1936        1
15                     Kim Tongni  1936        1
8                   Ch'ae Manshik  1938        1
6                  Hong Myŏng-hŭi  1939        0
20                 Ch'oe Chonghui  1948        1
12                   Hwang Sunwon  1953        1
16                     Yi Hoch'ol  1955        1
27                     Kim Sungok  1965        1
9                    Kim Pukhyang  1971        1


In [142]:
#######################################
# 5. Word Embeddings with Word2Vec
#######################################
# Word embeddings capture semantic relationships between words.
# In digital humanities, this helps analyze how the usage and connotations of certain terms evolve over time,
# offering insights into cultural and linguistic shifts.
# Tokenize texts for training the Word2Vec model
# df['tokens'] = df['lemmatized_tokens'].apply(nltk.word_tokenize)
w2v_model = Word2Vec(sentences=df['lemmatized_tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Example: Find words similar to 'tradition'
if 'tradition' in w2v_model.wv:
    print("\nWords similar to 'family':")
    print(w2v_model.wv.most_similar('family', topn=5))
else:
    print("\nThe word 'tradition' was not found in the vocabulary.")


Words similar to 'family':
[('without', 0.9998990893363953), ('well', 0.9998967051506042), ('still', 0.9998930096626282), ('son', 0.9998928308486938), ('people', 0.9998916983604431)]


In [145]:
#######################################
# 6. Named Entity Recognition (NER) with spaCy
#######################################
# NER extracts names, dates, places, and other entities from texts.
# This can be used to map historical figures, locations, and events mentioned in the translated texts,
# enriching the historical context and facilitating further research.
nlp = spacy.load("en_core_web_sm")
doc_example = nlp(df.loc[0, 'text'])
print("\nNamed Entities in the first document:")
for ent in doc_example.ents:
    print(ent.text, ent.label_)



Named Entities in the first document:
One CARDINAL
half CARDINAL
morning TIME
Shindang GPE
four CARDINAL
the new year DATE
61 CARDINAL
two CARDINAL
first ORDINAL
a month DATE
Sure! Take good care of it now WORK_OF_ART
every day DATE
today DATE
41 CARDINAL
62 CARDINAL
KIM YUJONG PERSON
almost a week DATE
one day DATE
Klimch'ol ORG
some day DATE
three or CARDINAL
four 
days DATE
63 CARDINAL
a night TIME
15 CARDINAL
Sugi LOC


In [146]:
df

,author,year,title,text,tokens,clean_tokens,lemmatized_tokens,cluster
0,Kim Yujong,1936,The White Rabbit,THE WHITE RABBIT \nNight and day I had that ra...,"[THE, WHITE, RABBIT, Night, and, day, I, had, ...","[WHITE, RABBIT, Night, day, rabbit, mind, want...","[WHITE, RABBIT, Night, day, rabbit, mind, want...",1
1,Yi T'aejun,1936,Crows,CROWS \nThe lamp was new and the glass shade d...,"[CROWS, The, lamp, was, new, and, the, glass, ...","[CROWS, lamp, new, glass, shade, yet, smell, k...","[CROWS, lamp, new, glass, shade, yet, smell, k...",1
2,Ch'oe Yun,1992,The Gray Snowman,THE GRAY SNOWMAN \nThe events of that period a...,"[THE, GRAY, SNOWMAN, The, events, of, that, pe...","[GRAY, SNOWMAN, events, period, almost, twenty...","[GRAY, SNOWMAN, event, period, almost, twenty,...",1
3,Yi Munyol,1979,The Old Hatter,THE OLD HATTER \nThe first encounter between o...,"[THE, OLD, HATTER, The, first, encounter, betw...","[OLD, HATTER, first, encounter, clan, urchins,...","[OLD, HATTER, first, encounter, clan, urchin, ...",1
4,Ch’oe Chansik,1914,The Shore,----- PAGE 151: Michael_J._Pettid_(editor)_-_S...,"[--, --, -, PAGE, 151, :, Michael_J._Pettid_, ...","[151, 145•, Shore, Haean, 해안, Chansik, 최찬식, Tr...","[151, 145•, Shore, Haean, 해안, Chansik, 최찬식, Tr...",0
5,Ch'oe Inho,1971,Another Man's Room,ANOTHER MAN'S ROOM \nHe had just come in from ...,"[ANOTHER, MAN, 'S, ROOM, He, had, just, come, ...","[ANOTHER, MAN, ROOM, come, street, tired, seem...","[ANOTHER, MAN, ROOM, come, street, tired, seem...",1
6,Hong Myŏng-hŭi,1939,Im Kkŏkchŏng,----- PAGE 227: Michael_J._Pettid_(editor)_-_S...,"[--, --, -, PAGE, 227, :, Michael_J._Pettid_, ...","[227, 221•, Im, Kkŏkchŏng, 임꺽정, Hong, Myŏng-hŭ...","[227, 221•, Im, Kkŏkchŏng, 임꺽정, Hong, Myŏng-hŭ...",0
7,Kim Yongha,1997,Lizard,LIZARD\nKiss the snake on the tongue. If it se...,"[LIZARD, Kiss, the, snake, on, the, tongue, .,...","[LIZARD, Kiss, snake, tongue, senses, fear, 'l...","[LIZARD, Kiss, snake, tongue, sens, fear, 'll,...",1
8,Ch'ae Manshik,1938,My Innocent Uncle,MY INNOCENT UNCLE \nMy uncle? You mean that fi...,"[MY, INNOCENT, UNCLE, My, uncle, ?, You, mean,...","[INNOCENT, UNCLE, uncle, mean, fine, gentleman...","[INNOCENT, UNCLE, uncle, mean, fine, gentleman...",1
9,Kim Pukhyang,1971,The Son,THE SON \nChun'gi came home after nearly a mon...,"[THE, SON, Chun'gi, came, home, after, nearly,...","[SON, Chun'gi, came, home, nearly, month, away...","[SON, Chun'gi, came, home, nearly, month, away...",1
